# AI and ML

## 1. TensorFlow

[TensorFlow](https://www.tensorflow.org/) is a machine learning environment. It can be used to used to identify and sort pictures as shown in the example below.

### 1.1. Image processing using TensorFlow

In [ ]:
## Install Python packages
# pip install tensorflow
# pip install tensorflow_hub

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import os
import csv

In [4]:
# Load the model directly from TensorFlow Hub with error handling
try:
    detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
except Exception as e:
    print(f"Error loading model: {e}")
    detector = None

2024-11-24 07:41:28.471852: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
def detect_humans(image_path):
    """Detects humans in a given image."""
    if not detector:
        print("Model not loaded. Cannot process images.")
        return False
    try:
        # Load the image
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Error: Could not load image at {image_path}")
            return False
        
        # Convert BGR to RGB 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Convert to float tensor
        input_tensor = tf.convert_to_tensor(image, dtype=tf.uint8)
        input_tensor = input_tensor[tf.newaxis, ...]
        
        # Run inference
        detections = detector(input_tensor)
        
        # Extract detection results
        classes = detections['detection_classes'][0].numpy().astype(np.int32)
        scores = detections['detection_scores'][0].numpy()
        
        # Check for humans (class ID 1 is person in COCO dataset)
        # Filter for high confidence detections
        human_indices = np.where((classes == 1) & (scores > 0.5))[0]
        
        return len(human_indices) > 0
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return False


In [6]:
def process_folder(folder_path, output_csv):
    """Process all images in a folder recursively and save results to CSV."""
    processed_count = 0
    human_count = 0
    
    # Open CSV file for writing
    with open(output_csv, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write header
        csv_writer.writerow(['Image Path'])
        
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Check if file is an image
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
                    full_path = os.path.join(root, file)
                    processed_count += 1
                    
                    try:
                        if detect_humans(full_path):
                            csv_writer.writerow([full_path])
                            human_count += 1
                    except Exception as e:
                        print(f"Error processing {full_path}: {e}")
    
    print(f"\nProcessed {processed_count} images.")
    print(f"Found humans in {human_count} images.")
    print(f"Results saved to {output_csv}")


In [7]:
# Example usage
image_folder = "../media/training-data/"
output_csv = "output/human_detection_results.csv"
process_folder(image_folder, output_csv)


Processed 6 images.
Found humans in 4 images.
Results saved to output/human_detection_results.csv


### 1.2. Use file location from text file to copy to another location 

In [1]:
import os
import shutil
import pandas as pd
from pathlib import Path

In [2]:
def copy_images(source_paths, destination_base):
    # Create destination directory if it doesn't exist
    os.makedirs(destination_base, exist_ok=True)
    
    # Keep track of copied files and any errors
    successful_copies = []
    failed_copies = []
    
    for source_path in source_paths:
        try:
            # Clean up the path and get the filename
            source_path = source_path.strip()
            filename = os.path.basename(source_path)
            
            # Create destination path
            destination_path = os.path.join(destination_base, filename)
            
            # Handle duplicate filenames by adding a number if necessary
            counter = 1
            while os.path.exists(destination_path):
                name, ext = os.path.splitext(filename)
                destination_path = os.path.join(destination_base, f"{name}_{counter}{ext}")
                counter += 1
            
            # Copy the file
            shutil.copy2(source_path, destination_path)
            successful_copies.append(filename)
            print(f"Successfully copied: {filename}")
            
        except Exception as e:
            failed_copies.append((filename, str(e)))
            print(f"Failed to copy {filename}: {str(e)}")
    
    # Print summary
    print("\nSummary:")
    print(f"Successfully copied {len(successful_copies)} files")
    if failed_copies:
        print(f"Failed to copy {len(failed_copies)} files:")
        for filename, error in failed_copies:
            print(f"- {filename}: {error}")

In [3]:
# Read file paths from CSV
csv_file = "output/human_detection_results.csv"
try:
    # Read the CSV file - assuming the paths are in the first column
    # If they're in a different column, you'll need to specify the column name
    df = pd.read_csv(csv_file)
    source_paths = df.iloc[:, 0].tolist()  # Gets the first column
except Exception as e:
    print(f"Error reading CSV file: {str(e)}")
    exit(1)

In [4]:
# Destination directory
destination_base = "destination/"

In [5]:
# Run the copy operation
copy_images(source_paths, destination_base)

Successfully copied: nathan-anderson-FHiJWoBodrs-unsplash.jpg
Successfully copied: ryoji-iwata-X53e51WfjlE-unsplash.jpg
Successfully copied: Watts_George_Frederick_Portrait_Of_The_Artists_Wife_Mary.jpg
Successfully copied: Alvígio_de_Barros_França_(dedé)_(1)_-_1-20725-0000-0000,_Acervo_do_Museu_Paulista_da_USP.jpg

Summary:
Successfully copied 4 files


## 2. Run your large language models locally with Ollama

Check the [docs of Ollama](https://github.com/ollama/ollama/tree/main/docs) to install Ollama locally. 

In [23]:
## After having installed Ollama you can install ollama packages for Python
## "ollama run llama3.2:1b" will pull and run a light version of llama3.2 
# pip install ollama

In [6]:
import os
import time
import ollama

In [7]:
# Define output file path
output_file = "output/image_analysis_results.txt"

In [ ]:
# Open the output file in append mode
with open(output_file, "a") as f:
  # Write header row
  f.write("Filename, Has People?, Execution Time (minutes)\n")

  # Get all image files in the ai-playground folder
  for filename in os.listdir("../media/training-data/"):
      image_path = os.path.join("../media/training-data/", filename)

      # Capture start time
      start_time = time.time()

      # Perform image analysis using Llama
      response = ollama.chat(
          model='llama3.2-vision',
          messages=[{
              'role': 'user',
              'content': 'Tell me whether there are people in this image. A simple yes/no is enough.',
              'images': [image_path]
          }]
      )

      # Calculate execution time
      end_time = time.time()
      execution_time = round((end_time - start_time) / 60, 2)

      # Extract people presence from response
      has_people = "Yes" if response["message"]["content"] == "Yes" else "No"

      # Write results to file
      f.write(f"{filename}, {has_people}, {execution_time}\n")

      print(f"Finished processing {filename}")

Finished processing nathan-anderson-FHiJWoBodrs-unsplash.jpg
Finished processing ryoji-iwata-X53e51WfjlE-unsplash.jpg
Finished processing Watts_George_Frederick_Portrait_Of_The_Artists_Wife_Mary.jpg
